In [ ]:
from lfp4uda.feature_extractor import FeatureExtractor
from netvlad import NetVLAD
from local_feature_alignment import LocalFeatureAlignment
from holistic_discriminator import HolisticDiscriminator
from local_discriminator import LocalDiscriminator
from single_layer_classifier import SingleLayerClassifier
from office_31_preprocessing import Office31
import numpy as np
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.python.data.ops.dataset_ops import MapDataset
import os

tf.autograph.set_verbosity(0)

In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
BATCH_SIZE = 32


In [ ]:
source, target = Office31().import_folder()[0:2] 
# Initialize cluster centroids with k means
x_source_batch, y_source_batch = next(iter(source))
x_target_batch, y_target_batch = next(iter(target))

feature_extractor_0 = FeatureExtractor()

input_0 = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
model_0 = tf.keras.models.Model(
    inputs=[input_0], outputs=feature_extractor_0(input_0))
features = model_0(x_source_batch)

features_np = tf.keras.backend.reshape(
    features,
    shape=(np.prod(features.shape[:3]), features.shape[3])
).numpy()

kmeans = KMeans(n_clusters=32)
kmeans.fit(features_np)
centroids = kmeans.cluster_centers_

# Step 1 : Classifier training

tf.keras.backend.clear_session()

## Defining layers
feature_extractor_1 = FeatureExtractor(False)
netvlad_1 = NetVLAD(5000, 0.005, centroids, False)
single_layer_classifier_1 = SingleLayerClassifier(31)
softmax_1 = tf.keras.layers.Softmax()

## Running step 1
image_input_1 = tf.keras.layers.Input(shape=(256, 256, 3))
layer1_1 = feature_extractor_1(image_input_1)
layer1_2, _, _ = netvlad_1(layer1_1)
layer1_3 = single_layer_classifier_1(layer1_2)
layer1_4 = softmax_1(layer1_3)
model_step_1 = tf.keras.models.Model(
    inputs=[image_input_1], outputs=layer1_4)

adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model_step_1.compile(loss='categorical_crossentropy', optimizer=adam)
# model_step_1(x_source_batch)
model_step_1.fit(x_source_batch, y_source_batch, epochs=1)

print("Step 1 succeed")


In [ ]:
# Step 2 : Source finetuning

## Defining layers
feature_extractor_2 = FeatureExtractor(True)
netvlad_2 = NetVLAD(5000, 0.005, netvlad_1.centers.numpy(), True)
single_layer_classifier_2 = single_layer_classifier_1
softmax_2 = softmax_1

## Running step 2
image_input_2 = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
features_2 = feature_extractor_2(image_input_2)
vlad_2, similarities_2, _ = netvlad_2(features_2)
class_2 = single_layer_classifier_2(
    softmax_2(vlad_2))

model_step_2 = tf.keras.models.Model(
    inputs=[image_input_2],
    outputs=[class_2, similarities_2]
)
def loss_2(y_true, y_pred):
    print(tf.shape(y_pred))
    return 1.0
    
adam_2 = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_step_2.compile(loss=loss_2, optimizer=adam_2)
# model_step_2.fit(x_source_batch, y_source_batch, epochs=1)
# Step 3 : Domain adaptation


In [11]:
print(IMG_WEIGHT)

NameError: name 'feature_extractor' is not defined

In [ ]:
feature_extractor.vgg16.summary()

In [ ]:
feature_extractor_test = FeatureExtractor(True)

feature_extractor_test.vgg16.summary()

# ones = tf.constant([[1.0, 2.0],[3.0, 4.0]])
# print(-tf.keras.backend.mean(ones))

In [ ]:
labels = tf.constant([True, False, False])
def f(x):
    if x:
        return 1.0
    return 0.0

print(tf.keras.backend.map_fn(f, labels))